# Load Data

In [27]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.graph_objects as go

import DataRetriever as dr

retriever = dr.DataRetriever()

PV_ATTRIBUTES = retriever.get_attributes(file_name='producing_attributes.pkl')

gen_df = retriever.get_data(file_name='All-Subsystems-hour-Year2.pkl')[PV_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000
gen_df = gen_df.rename("Generated Energy", inplace=True).to_frame()

weather_forecast = pd.read_csv("../_05Forecasting/CLEANED_GAI_2015_2016.csv", index_col=0)
weather_forecast.index.rename("Timestamp", inplace=True)
weather_forecast.index = pd.to_datetime(weather_forecast.index)

# OneHotEncode the weather data

In [28]:
weather_forecast = weather_forecast[weather_forecast.index.isin(gen_df.index)]

ohe = OneHotEncoder(sparse=False)
hot = ohe.fit_transform(weather_forecast)
weather_df = pd.DataFrame(data=hot, columns=ohe.get_feature_names_out(), index=gen_df.index)

In [29]:
data = gen_df.merge(weather_df, left_index=True, right_index=True, how='left')
condition_columns = data.columns[1:]

In [30]:
train_val = data[: data.index[len(data) * 0.8]]
test = data[data.index[len(data) * 0.8]: ]

/tmp/ipykernel_128208/3376622727.py:1: FutureWarning: Indexing with a float is deprecated, and will raise an IndexError in pandas 2.0. You can manually convert to an integer key instead.
  train_val = data[: data.index[len(data) * 0.8]]
/tmp/ipykernel_128208/3376622727.py:2: FutureWarning: Indexing with a float is deprecated, and will raise an IndexError in pandas 2.0. You can manually convert to an integer key instead.
  test = data[data.index[len(data) * 0.8]: ]


# Rolling Window Function

In [31]:
def sliding_forecast(endog: pd.DataFrame, trend: str, order: tuple,
                     seasonal_order: tuple=None, exog: pd.DataFrame=None,
                     train_length: int=0, val_length: int=0, shift: int=0):
    train_start = 0
    train_end = train_start + (train_length * 24)
    val_start = train_end
    val_end = val_start + (val_length * 24)
    progress = 0

    rmse_list = list()

    while len(endog[: val_end]) < len(endog):
        if exog is not None:
            model = SARIMAX(endog=endog[train_start : train_end],
                exog=exog[train_start : train_end],
                trend=trend,
                order=order,
                seasonal_order=seasonal_order)\
                .fit(low_memory=True, disp=False, full_output=False)

            predictions = pd.DataFrame(model.predict(start=endog.index[val_start],
                                                     end=endog.index[val_end - 1],
                                                     exog=exog[val_start: val_end]))

        else:
            model = SARIMAX(endog=endog[train_start : train_end],
                trend=trend,
                order=order,
                seasonal_order=seasonal_order)\
                .fit(low_memory=True, disp=False, full_output=False)

            predictions = pd.DataFrame(model.predict(start=endog.index[val_start],
                                                     end=endog.index[val_end - 1]))

        rmse_list.append(mean_squared_error(y_true=endog[val_start: val_end],
                                            y_pred=predictions,
                                            squared=False))

        train_start += shift * 24
        train_end = train_start + (train_length * 24)
        val_start = train_end
        val_end = val_start + (val_length * 24)

        progress += 1

    print(progress)

    return sum(rmse_list) / len(rmse_list)

In [32]:
ARIMA_RMSE = sliding_forecast(endog=test["Generated Energy"],
                 trend='n', order=(4, 0, 0),
                 train_length=28, val_length=3, shift=5)

9


In [33]:
print(f"RMSE for ARIMA(4, 0, 0) is: {round(ARIMA_RMSE, 5)}")

RMSE for ARIMA(4, 0, 0) is: 1.7122


In [34]:
sarima_params = [(4, 0, 0, 2, 0, 0, 24), (4, 1, 0, 2, 0, 0, 24),
                 (4, 0, 0, 2, 1, 0, 24), (4, 1, 0, 2, 1, 0, 24),
                 (4, 0, 0, 0, 0, 2, 24), (4, 1, 0, 0, 0, 2, 24),
                 (4, 0, 0, 0, 1, 2, 24), (4, 1, 0, 0, 1, 2, 24)]

sarima_rmses = dict()
key = 1

for params in sarima_params:
    result = sliding_forecast(endog=test["Generated Energy"],
                              trend='n', order=params[0: 3], seasonal_order=params[3: ],
                              train_length=28, val_length=3, shift=5)
    sarima_rmses[key] = result
    print(f"RMSE for SARIMA{params} is: {round(result, 5)}")
    key += 1

9
RMSE for SARIMA(4, 0, 0, 2, 0, 0, 24) is: 1.57232
9
RMSE for SARIMA(4, 1, 0, 2, 0, 0, 24) is: 1.57494
9
RMSE for SARIMA(4, 0, 0, 2, 1, 0, 24) is: 1.65527
9
RMSE for SARIMA(4, 1, 0, 2, 1, 0, 24) is: 1.65554
9
RMSE for SARIMA(4, 0, 0, 0, 0, 2, 24) is: 1.65498
9
RMSE for SARIMA(4, 1, 0, 0, 0, 2, 24) is: 1.64789
9
RMSE for SARIMA(4, 0, 0, 0, 1, 2, 24) is: 1.49988
9
RMSE for SARIMA(4, 1, 0, 0, 1, 2, 24) is: 1.50208


In [ ]:
best_rmse = min(sarima_rmses.values())
best_param_key = [key for key in sarima_rmses if sarima_rmses[key] == best_rmse]
best_params = sarima_params[best_param_key[0]] #Access index 0 as the best_param_key returns a list

SARIMAX_RMSE = sliding_forecast(endog=test["Generated Energy"], exog=test[condition_columns],
                 trend='n', order=best_params[0: 3], seasonal_order=best_params[3: ],
                 train_length=28, val_length=3, shift=5)

print(round(SARIMAX_RMSE, 5))

9
1.5936


# Plot the predictions from a single window of the best model

In [36]:
start_train = 0
end_train = start_train + 24*28
start_test = end_train
end_test = start_test + 3*24

In [40]:
start_test, end_test

(672, 744)

In [41]:
test.index[672: 744]

DatetimeIndex(['2015-12-18 00:00:00', '2015-12-18 01:00:00',
               '2015-12-18 02:00:00', '2015-12-18 03:00:00',
               '2015-12-18 04:00:00', '2015-12-18 05:00:00',
               '2015-12-18 06:00:00', '2015-12-18 07:00:00',
               '2015-12-18 08:00:00', '2015-12-18 09:00:00',
               '2015-12-18 10:00:00', '2015-12-18 11:00:00',
               '2015-12-18 12:00:00', '2015-12-18 13:00:00',
               '2015-12-18 14:00:00', '2015-12-18 15:00:00',
               '2015-12-18 16:00:00', '2015-12-18 17:00:00',
               '2015-12-18 18:00:00', '2015-12-18 19:00:00',
               '2015-12-18 20:00:00', '2015-12-18 21:00:00',
               '2015-12-18 22:00:00', '2015-12-18 23:00:00',
               '2015-12-19 00:00:00', '2015-12-19 01:00:00',
               '2015-12-19 02:00:00', '2015-12-19 03:00:00',
               '2015-12-19 04:00:00', '2015-12-19 05:00:00',
               '2015-12-19 06:00:00', '2015-12-19 07:00:00',
               '2015-12-

In [42]:
model = SARIMAX(endog=test["Generated Energy"][test.index[start_train: end_train]],
                exog=test[condition_columns][start_train : end_train],
                trend='n',
                order=(4,0,0),
                seasonal_order=(0,1,2,24))\
    .fit(low_memory=True, disp=False, full_output=False)

predictions = pd.DataFrame(model.predict(start=test.index[start_test],
                                         end=test.index[end_test],
                                         exog=test[condition_columns][start_test: end_test + 1]))

In [50]:
test.index[start_train]

Timestamp('2015-11-20 00:00:00', freq='H')

In [51]:
fig = go.Figure(go.Scattergl(
    x=test[start_train: end_train].index,
    y=test['Generated Energy'][start_train: end_train],
    name="Training Data",
    line=dict(color='rgb(84, 84, 84)')
))

fig.add_trace(go.Scattergl(
    x=test[start_test: end_test].index,
    y=test['Generated Energy'][start_test: end_test],
    name="Observed",
    line=dict(color='rgb(234,143,129)')
))

fig.add_trace(go.Scattergl(
    x=predictions.index,
    y=predictions['predicted_mean'],
    name="Predicted",
    line=dict(color='rgb(32,115,171)')
))

fig.update_xaxes(range=['2015-11-20 00:00:00', '2015-12-21 00:00:00'])
fig.update_yaxes(title="Generated Energy [kWh]")
fig.update_layout(template="plotly",
                  legend=dict(orientation="h",
                              yanchor="bottom",
                              y=1,
                              xanchor="left",
                              x=0))

fig.write_html("slidingwindowarima/window_sarimax_producing_NEW.html")
fig.show()